# Individual assignment Data mining 
### National Health and Nutrition Examination Survey

For this assignment we have a set of 6 different files with data available from survey research. 

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, confusion_matrix, classification_report, 
                             silhouette_score)
from sklearn.preprocessing import MinMaxScaler


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mlxtend.frequent_patterns import fpgrowth, association_rules


## Importing the data

In [2]:
df_diet = pd.read_csv("data/diet.csv")

In [3]:
[col for col in df_diet.columns if col.startswith("DR1")]

['DR1DRSTZ',
 'DR1EXMER',
 'DR1DBIH',
 'DR1DAY',
 'DR1LANG',
 'DR1MNRSP',
 'DR1HELPD',
 'DR1STY',
 'DR1SKY',
 'DR1TNUMF',
 'DR1TKCAL',
 'DR1TPROT',
 'DR1TCARB',
 'DR1TSUGR',
 'DR1TFIBE',
 'DR1TTFAT',
 'DR1TSFAT',
 'DR1TMFAT',
 'DR1TPFAT',
 'DR1TCHOL',
 'DR1TATOC',
 'DR1TATOA',
 'DR1TRET',
 'DR1TVARA',
 'DR1TACAR',
 'DR1TBCAR',
 'DR1TCRYP',
 'DR1TLYCO',
 'DR1TLZ',
 'DR1TVB1',
 'DR1TVB2',
 'DR1TNIAC',
 'DR1TVB6',
 'DR1TFOLA',
 'DR1TFA',
 'DR1TFF',
 'DR1TFDFE',
 'DR1TCHL',
 'DR1TVB12',
 'DR1TB12A',
 'DR1TVC',
 'DR1TVD',
 'DR1TVK',
 'DR1TCALC',
 'DR1TPHOS',
 'DR1TMAGN',
 'DR1TIRON',
 'DR1TZINC',
 'DR1TCOPP',
 'DR1TSODI',
 'DR1TPOTA',
 'DR1TSELE',
 'DR1TCAFF',
 'DR1TTHEO',
 'DR1TALCO',
 'DR1TMOIS',
 'DR1TS040',
 'DR1TS060',
 'DR1TS080',
 'DR1TS100',
 'DR1TS120',
 'DR1TS140',
 'DR1TS160',
 'DR1TS180',
 'DR1TM161',
 'DR1TM181',
 'DR1TM201',
 'DR1TM221',
 'DR1TP182',
 'DR1TP183',
 'DR1TP184',
 'DR1TP204',
 'DR1TP205',
 'DR1TP225',
 'DR1TP226',
 'DR1.300',
 'DR1.320Z',
 'DR1.330Z',
 'DR1BWATZ',

In [4]:
df_diet["DR1CCMTX"]

KeyError: 'DR1CCMTX'

# 

# Analyzing the vegatarians
 order for us to analyze the vegatarians in the data, we must find the vegatarians. However, in the data there is no self reported vegatarian question. However, what is available is in a .xpt file on https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/DS1IDS_H.htm. In this file we can see what each user who is represented by a unique SEQN, what their daily food consisted out of. There are two separate data files, where we can find the food that has been eaten by the participant. DR1IFF_H.xpt  includes the food data of day 1 and DR2IFF_H.xpt contains the food data of day 2. Not every participant in the provided 5 data files from kaggle, has participated in the food eating evaluation interviews. So we will only take the rows where the sequence number exists in both food evaluating interviews and also in the general interview. 
 
In order to label the SEQN as vegetarian or non_vegatarian, we must implement a proxy to identify a vegetarian. As proxy we use whether an individual has eaten no meat, poultry or fish. 

In [5]:
df_day1 = pd.read_sas("data/DR1IFF_H.xpt")

In [6]:
print(f"A total of {len(df_day1.SEQN.unique())} unique SEQN numbers are in the dataset")
print(f"A total of {len(df_day1)} rows are in the dataset, which means that each individual has reported {round(len(df_day1)/len(df_day1.SEQN.unique()), ndigits=1)} different food items on day 1")

A total of 8661 unique SEQN numbers are in the dataset
A total of 131394 rows are in the dataset, which means that each individual has reported 15.2 different food items on day 1


In the day 1 dataset, there are 8661 different participants who have reported 15.2 different food items for a total of 13194 total food items in day 1. 

We now want to evaluate the food items efficiently, we can use a metric which is the food group item namely "DR1CCMTX". This metric devides all possible food items in into 16 groups. Below you can see the results. We see that there is an error, the extreme negative value (5.4 * e-79), should be 0, but this might be due to the xpt reading of the food. So we should round the vlaues of DR1CCMTX

In [7]:
df_day1.DR1CCMTX.unique()

array([5.39760535e-79, 1.00000000e+00, 9.00000000e+01, 2.00000000e+00,
       3.00000000e+00, 5.00000000e+00, 1.10000000e+01, 1.00000000e+01,
       9.00000000e+00, 1.20000000e+01, 4.00000000e+00, 1.40000000e+01,
       6.00000000e+00, 8.00000000e+00, 1.30000000e+01, 7.00000000e+00])

In [8]:
df_day1["DR1CCMTX"] = round(df_day1["DR1CCMTX"])
occurrences = df_day1["DR1CCMTX"].value_counts().sort_index()
occurrences

DR1CCMTX
0.0     75335
1.0     10947
2.0      5466
3.0      5300
4.0      5337
5.0     13313
6.0       832
7.0        52
8.0       491
9.0      3690
10.0      666
11.0     1781
12.0     2653
13.0      169
14.0      721
90.0     4641
Name: count, dtype: int64

In [9]:
len(df_day1[df_day1["DR1CCMTX"] == 0]["SEQN"].unique())

8648

## First found challenge
The food combination type metric "DR1CCMTX", includes a lot of 0 values. The value 0, refers to the food combination not having a specific code in the researchers coding system. We thought we might be able to be smart and exclude the participants that have eaten food that has food code 0. But this group consists 99.9% of the dataset, so we have to find a workaround. As there are only 721 instances of food that participants have eaten food that belongs to the poultry, meat and fish group, which is number 14

You can see a table that includes the foodtypes and their respective code below

![IMG](Images/Table-food-type-codes.png)
source:  https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2013/DataFiles/DRXFCD_H.htm

## How to label vegetarians

As the group of combination food codes has 57% undefined values, who might contain meat or fish. We must use a different approach to find out who in this dataset is vegetarian. There is another column that identifies what the food type can be in this dataset. It is the column DR1IFDCD, which has a column description of the USDA food code of the food, (called DR2IFDCD in day 2 dataset). So by identifying which DR1IFDCD food codes contain meat, poultry or fish, we should be able to correctly link participants to being vegetarian. 

The only missing piece that is left is to find out which food codes contain meat and which do not. Luckily, there is a description of food codes file available at the 

In [10]:
df_day1

,SEQN,WTDRD1,WTDR2D,DR1ILINE,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,...,DR1IM181,DR1IM201,DR1IM221,DR1IP182,DR1IP183,DR1IP184,DR1IP204,DR1IP205,DR1IP225,DR1IP226
0,73557.0,16888.327864,12930.890649,1.0,1.0,49.0,2.0,2.0,6.0,2.0,...,3.595000e+00,3.400000e-02,1.000000e-03,9.490000e-01,1.080000e-01,5.397605e-79,5.100000e-02,1.000000e-03,5.397605e-79,1.000000e-02
1,73557.0,16888.327864,12930.890649,2.0,1.0,49.0,2.0,2.0,6.0,2.0,...,5.397605e-79,5.397605e-79,5.397605e-79,4.000000e-03,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
2,73557.0,16888.327864,12930.890649,3.0,1.0,49.0,2.0,2.0,6.0,2.0,...,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
3,73557.0,16888.327864,12930.890649,4.0,1.0,49.0,2.0,2.0,6.0,2.0,...,8.100000e-02,5.397605e-79,5.397605e-79,1.030000e-01,3.100000e-02,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
4,73557.0,16888.327864,12930.890649,5.0,1.0,49.0,2.0,2.0,6.0,2.0,...,2.600000e-02,5.397605e-79,5.397605e-79,2.400000e-02,9.000000e-03,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131389,83731.0,5805.674812,4339.132077,23.0,1.0,49.0,2.0,2.0,12.0,6.0,...,3.798000e+00,3.800000e-02,5.397605e-79,3.372000e+00,4.790000e-01,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
131390,83731.0,5805.674812,4339.132077,24.0,1.0,49.0,2.0,2.0,12.0,6.0,...,5.260000e-01,5.000000e-03,5.397605e-79,4.730000e-01,8.200000e-02,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
131391,83731.0,5805.674812,4339.132077,25.0,1.0,49.0,2.0,2.0,12.0,6.0,...,1.483000e+00,1.500000e-02,5.397605e-79,1.346000e+00,1.980000e-01,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79
131392,83731.0,5805.674812,4339.132077,26.0,1.0,49.0,2.0,2.0,12.0,6.0,...,6.830000e-01,5.397605e-79,5.397605e-79,6.000000e-02,2.500000e-02,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79


In [11]:
df_foodcodes_description = pd.read_sas("data/DRXFCD_H.xpt")

In [12]:
df_foodcodes_description["DRXFDCD"] = df_foodcodes_description["DRXFDCD"].apply(lambda x: int(x))
df_foodcodes_description

,DRXFDCD,DRXFCSD,DRXFCLD
0,11000000,"b'MILK, HUMAN'","b'Milk, human'"
1,11100000,"b'MILK, NFS'","b'Milk, NFS'"
2,11111000,"b'MILK, WHOLE'","b'Milk, whole'"
3,11111100,"b'MILK, LOW SODIUM, WHOLE'","b'Milk, low sodium, whole'"
4,11111150,"b'MILK, CALCIUM FORTIFIED, WHOLE'","b'Milk, calcium fortified, whole'"
...,...,...,...
8531,95323000,"b'SPORTS DRINK, LOW CALORIE'","b'Sports drink, low calorie'"
8532,95330100,"b'FLUID REPLACEMENT, ELECTROLYTE SOLUTION'","b'Fluid replacement, electrolyte solution'"
8533,95330500,"b'FLUID REPLACEMENT, 5% GLUCOSE IN WATER'","b'Fluid replacement, 5% glucose in water'"
8534,95341000,b'FUZE SLENDERIZE FORTIFIED LOW CALORIE FRUIT ...,b'FUZE Slenderize fortified low calorie fruit ...


In [13]:
def turn_binary_string_to_list(df):
    df["food_list"] = df["DRXFCSD"].apply(lambda x: str(x).replace("b'", "").replace("'", "").split(","))
    return df
df_foodcodes_description = turn_binary_string_to_list(df_foodcodes_description)

In [16]:
def turn_food_df_to_dict(df):
    food_code_dict = {}
    df_food_code = df[["DRXFDCD", "food_list"]]
    for index, row in df_food_code.iterrows():
        food_code_dict[row["DRXFDCD"]] = row["food_list"]
    return food_code_dict
food_code_dict = turn_food_df_to_dict(df_foodcodes_description)

In [15]:
import LLMConnect
import json
def turn_json_into_segments(json_to_divide, dict_per_chunk=50):
    

    smaller_json = {}
    json_list_combined = []
    for index,(key, json_dict) in enumerate(json_to_divide.items()):
        if index % dict_per_chunk ==0 and index !=0:
            json_list_combined.append(smaller_json)
            smaller_json = {}
        smaller_json[key] = json_dict
    json_list_combined.append(smaller_json)
    return json_list_combined
    


def detect_food_json_list_dict(json_list:str):
    json_list = turn_json_into_segments(json_to_divide=json_list,
                                        dict_per_chunk=50)    
    
    
    
    LLM_translator = LLMConnect.DetectFoodIngredients(model_name="gpt-4o-mini", max_tokens=9069)
    
    complete_food_labeled_json_dict = []
    
    for index, json_dict in enumerate(json_list):
        food_json= json.dumps(json_dict)
        food_labeled_json = LLM_translator.detect_food_ingredients(input_json= food_json
                                                               )
        
        complete_food_labeled_json_dict.append(food_labeled_json)
    return complete_food_labeled_json_dict
        
        
complete_food_json = detect_food_json_list_dict(json_list=food_code_dict)



/var/folders/f3/ykt1vfz96xz48try_trwdn440000gn/T/ipykernel_44914/4058991999.py:24: UserWarning: WARNING! response_format is not default parameter.
                response_format was transferred to model_kwargs.
                Please confirm that response_format is what you intended.
  LLM_translator = LLMConnect.DetectFoodIngredients(model_name="gpt-4o-mini", max_tokens=9069)


I am sending a request to an LLM now
save_file_path =  json/data.json
save_file_path =  json/data.json
I am sending a request to an LLM now
save_file_path =  json/data_0.json
save_file_path =  json/data_0.json


In [27]:
len(complete_food_json[1])

50

In [42]:
len(complete_food_json["ingredients"])

KeyError: 'ingredients'

In [23]:
complete_food_json

[{'meals': [{'ingredient_list': ['MILK', 'HUMAN'],
    'meal_number': '11000000',
    'poultry': 0,
    'red_meat': 0,
    'fish': 0,
    'shellfish': 0,
    'dairy_products': 1,
    'other_animal_products': 0,
    'none': 0},
   {'ingredient_list': ['MILK', 'NFS'],
    'meal_number': '11100000',
    'poultry': 0,
    'red_meat': 0,
    'fish': 0,
    'shellfish': 0,
    'dairy_products': 1,
    'other_animal_products': 0,
    'none': 0},
   {'ingredient_list': ['MILK', 'WHOLE'],
    'meal_number': '11111000',
    'poultry': 0,
    'red_meat': 0,
    'fish': 0,
    'shellfish': 0,
    'dairy_products': 1,
    'other_animal_products': 0,
    'none': 0},
   {'ingredient_list': ['MILK', 'LOW SODIUM', 'WHOLE'],
    'meal_number': '11111100',
    'poultry': 0,
    'red_meat': 0,
    'fish': 0,
    'shellfish': 0,
    'dairy_products': 1,
    'other_animal_products': 0,
    'none': 0},
   {'ingredient_list': ['MILK', 'CALCIUM FORTIFIED', 'WHOLE'],
    'meal_number': '11111150',
    'poultry'

In [28]:
complete_food_json

{11000000: ['MILK', ' HUMAN'],
 11100000: ['MILK', ' NFS'],
 11111000: ['MILK', ' WHOLE'],
 11111100: ['MILK', ' LOW SODIUM', ' WHOLE'],
 11111150: ['MILK', ' CALCIUM FORTIFIED', ' WHOLE'],
 11111160: ['MILK', ' CALCIUM FORTIFIED', ' LOW FAT (1%)'],
 11111170: ['MILK', ' CALCIUM FORTIFIED', ' FAT FREE (SKIM)'],
 11112110: ['MILK', ' REDUCED FAT (2%)'],
 11112120: ['MILK', ' ACIDOPHILUS', ' LOW FAT (1%)'],
 11112130: ['MILK', ' ACIDOPHILUS', ' REDUCED FAT (2%)'],
 11112210: ['MILK', ' LOW FAT (1%)'],
 11113000: ['MILK', ' FAT FREE (SKIM)'],
 11114300: ['MILK', ' LACTOSE FREE', ' LOW FAT (1%)'],
 11114320: ['MILK', ' LACTOSE FREE', ' FAT FREE (SKIM)'],
 11114330: ['MILK', ' LACTOSE FREE', ' REDUCED FAT (2%)'],
 11114350: ['MILK', ' LACTOSE FREE', ' WHOLE'],
 11115000: ['BUTTERMILK', ' FAT FREE (SKIM)'],
 11115100: ['BUTTERMILK', ' LOW FAT (1%)'],
 11115200: ['BUTTERMILK', ' REDUCED FAT (2%)'],
 11115300: ['BUTTERMILK', ' WHOLE'],
 11115400: ['KEFIR', ' NS AS TO FAT CONTENT'],
 11116000: 

In [104]:
unique_elemnts_food = set([food for food_list in df_foodcodes_description.food_list.values for food in food_list])

In [105]:
len(unique_elemnts_food)

6191

All different foods in the food description column are 

In [76]:
95330100 in df_foodcodes_description.DRXFDCD.unique()

True

In [27]:
for index, row in df_foodcodes_description.iterrows():
    if "meat" in str(row["DRXFCLD"]):
        print(row)

DRXFDCD                    14620320.0
DRXFCSD    b'TOPPING FROM MEAT PIZZA'
DRXFCLD    b'Topping from meat pizza'
Name: 724, dtype: object
DRXFDCD                                  14620330.0
DRXFCSD    b'TOPPING FROM MEAT AND VEGETABLE PIZZA'
DRXFCLD    b'Topping from meat and vegetable pizza'
Name: 725, dtype: object
DRXFDCD                                          20000090.0
DRXFCSD         b'MEAT STICKS, BABY, NS AS TO TYPE OF MEAT'
DRXFCLD    b'Meat sticks, baby food, NS as to type of meat'
Name: 741, dtype: object
DRXFDCD             20000200.0
DRXFCSD     b'GROUND MEAT,NFS'
DRXFCLD    b'Ground meat, NFS'
Name: 742, dtype: object
DRXFDCD                                           21001000.0
DRXFCSD    b'STEAK, NS AS TO TYPE OF MEAT, COOKED, NS AS ...
DRXFCLD    b'Steak, NS as to type of meat, cooked, NS as ...
Name: 746, dtype: object
DRXFDCD                                           21001010.0
DRXFCSD    b'STEAK, NS AS TO TYPE OF MEAT, COOKED, LEAN &...
DRXFCLD    b'Steak, NS as t